<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Model IO Exercise 

The purpose of this exercise is to test your understanding of building out Model IO systems. You will also hopefully notice the need to chain responses together, which we will cover later in this course!

Watch the video for a full overview on minimum outputs this class should be capable of, but feel free to expand on this project, or to just treat it as a code-along!

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

### Suggested Imports

Feel free to accomplish this task however you prefer!

In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
# from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv
import os
load_dotenv()
KEY = os.getenv("API_KEY")

from langchain_openai import ChatOpenAI
model = ChatOpenAI(api_key=KEY)

In [26]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        system_template = "You are a tutor in history subject. You will give one question about the topic that requested by the user that has a date as the correct answer."
        system_prompt = SystemMessagePromptTemplate.from_template(system_template)

        human_template = "{topic}"
        human_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])
        request = chat_prompt.format_prompt(topic = topic).to_messages()
        response = model.invoke(request)
        question = response.content
        return question
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        dt_parser = DatetimeOutputParser()
        human_template = "{question}\n{format_instructions}"
        human_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages([human_prompt])
        request = chat_prompt.format_prompt(question=question, format_instructions=dt_parser.get_format_instructions()).to_messages()
        response = model.invoke(request)
        correct_datetime = dt_parser.parse(response.content)

        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(f"{question}?")
        input_year = "1950"
        input_month = "6"
        input_date = "22"
        dt_answer = f"{input_year}-{input_month.zfill(2)}-{input_date.zfill(2)}"
        user_datetime = datetime.strptime(dt_answer,"%Y-%m-%d") 
        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        diff = user_answer - ai_answer
        print(diff)
        # print or return the difference between the answers here!
        pass
        

### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [27]:
quiz_bot = HistoryQuiz()

In [9]:
question = quiz_bot.create_history_question(topic='World War 2')

In [10]:
question

'Question: On what date did World War II begin?'

In [28]:
ai_answer = quiz_bot.get_AI_answer(question)

In [29]:
# Day After Pearl Harbor
ai_answer

datetime.datetime(1939, 9, 1, 0, 0)

In [30]:
user_answer = quiz_bot.get_user_answer(question)

Question: On what date did World War II begin??


In [31]:
user_answer

datetime.datetime(1950, 6, 22, 0, 0)

In [32]:
quiz_bot.check_user_answer(user_answer,ai_answer)

3947 days, 0:00:00
